# LOAD THE TRAIN AND TEST DATASETS

In [83]:
import pandas as pd
import numpy as np

In [84]:
x_train=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_train_clean.csv")
x_test=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_test_clean.csv")

In [85]:
#LOAD Y_TRAIN AND Y_TEST
y_train=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\y_train.csv")
y_test=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\y_test.csv")

In [86]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(246008, 137)
(61503, 137)
(246008, 1)
(61503, 1)


In [87]:
print(x_train.head())

  NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0         Cash loans           F            N               Y             0   
1         Cash loans           M            N               Y             0   
2         Cash loans           F            Y               Y             0   
3         Cash loans           M            Y               Y             0   
4         Cash loans           F            N               Y             0   

   AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE NAME_TYPE_SUITE  \
0           72000.0    246357.0      24493.5         234000.0   Unaccompanied   
1           90000.0    555273.0      21645.0         463500.0          Family   
2          292500.0   1006920.0      48573.0         900000.0        Children   
3          225000.0    500490.0      53901.0         450000.0   Unaccompanied   
4          135000.0    755190.0      32125.5         675000.0          Family   

   ... COMMONAREA_MODE_missing_flag FL

## MERGE X_TRAIN AND Y_TRAIN
BECAUSE WOE BINNING NEEDS TARGET VARIABLE TO CALCULATE WOE

In [88]:
train_data=x_train.copy(deep=True)

In [89]:
train_data['TARGET']=y_train

In [90]:
#shape of train_Data
train_data.shape

(246008, 138)

In [91]:
train_data.columns

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
       ...
       'FLOORSMIN_MODE_missing_flag', 'LIVINGAPARTMENTS_MODE_missing_flag',
       'NONLIVINGAPARTMENTS_MODE_missing_flag',
       'YEARS_BUILD_MEDI_missing_flag', 'COMMONAREA_MEDI_missing_flag',
       'FLOORSMIN_MEDI_missing_flag', 'LIVINGAPARTMENTS_MEDI_missing_flag',
       'NONLIVINGAPARTMENTS_MEDI_missing_flag',
       'FONDKAPREMONT_MODE_missing_flag', 'TARGET'],
      dtype='object', length=138)

In [92]:
train_data.isnull().sum().sum()

np.int64(0)

In [93]:
train_data['TARGET'].value_counts()

TARGET
0    226148
1     19860
Name: count, dtype: int64

# SEPARATE NUMERICAL COLS CATEGORICAL COLS AND MISSING FLAG COLS

In [94]:
pd.set_option('display.max_info_columns',100000)

In [95]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246008 entries, 0 to 246007
Data columns (total 138 columns):
 #    Column                                 Non-Null Count   Dtype  
---   ------                                 --------------   -----  
 0    NAME_CONTRACT_TYPE                     246008 non-null  object 
 1    CODE_GENDER                            246008 non-null  object 
 2    FLAG_OWN_CAR                           246008 non-null  object 
 3    FLAG_OWN_REALTY                        246008 non-null  object 
 4    CNT_CHILDREN                           246008 non-null  int64  
 5    AMT_INCOME_TOTAL                       246008 non-null  float64
 6    AMT_CREDIT                             246008 non-null  float64
 7    AMT_ANNUITY                            246008 non-null  float64
 8    AMT_GOODS_PRICE                        246008 non-null  float64
 9    NAME_TYPE_SUITE                        246008 non-null  object 
 10   NAME_INCOME_TYPE                       246

In [96]:
numerical_cols=train_data.select_dtypes(include=['int64','float64']).columns.tolist()
print(numerical_cols)

['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAR

In [97]:
#remove target from numerical cols and missing flag cols
numerical_cols.remove('TARGET')

In [98]:
#remove missing flag cols
missing_flag_cols=[i for i in numerical_cols if '_missing_flag' in i]
# numerical_cols=[i for i in numerical_cols if '_missing_flag' in i==False]
# print(missing_flag_cols)

In [99]:
numerical_cols=[i for i in numerical_cols if ('_missing_flag' not in i)]

In [100]:
numerical_cols

['CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXP

In [101]:
#get categorical column
categorical_cols=train_data.select_dtypes(include='object').columns.tolist()

In [102]:
categorical_cols

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [103]:
print("Numerical cols ", len(numerical_cols))
print("Categorical cols ", len(categorical_cols))
print("Flag cols ", len(missing_flag_cols))

Numerical cols  104
Categorical cols  16
Flag cols  17


# CREATE BIN FOR ONE NUMERICAL VARIABLE
EXT_SOURCE_2

In [104]:
train_data['EXT_SOURCE_2_BIN']=pd.qcut(train_data['EXT_SOURCE_2'],q=10)

In [105]:
EXT_SOURCE_2_BIN_CROSSTAB=pd.crosstab(index=train_data['EXT_SOURCE_2_BIN'],columns=train_data['TARGET'])

In [106]:
EXT_SOURCE_2_BIN_CROSSTAB

TARGET,0,1
EXT_SOURCE_2_BIN,,
"(-0.0009999183, 0.217]",20096,4514
"(0.217, 0.341]",21612,2980
"(0.341, 0.441]",22156,2446
"(0.441, 0.513]",22530,2072
"(0.513, 0.566]",22997,1865
"(0.566, 0.608]",22741,1596
"(0.608, 0.646]",23156,1444
"(0.646, 0.682]",23417,1187
"(0.682, 0.722]",23565,1033


In [107]:
#total defaults and non defaults
EXT_SOURCE_2_BIN_CROSSTAB.apply(lambda x: x.sum(), axis=0)

TARGET
0    226148
1     19860
dtype: int64

## Create two new columns:
dist_non_default
dist_default

In [108]:
EXT_SOURCE_2_BIN_CROSSTAB[['dist_non_default','dist_default']]=EXT_SOURCE_2_BIN_CROSSTAB.apply(lambda x: x/x.sum(), axis=0)
EXT_SOURCE_2_BIN_CROSSTAB

TARGET,0,1,dist_non_default,dist_default
EXT_SOURCE_2_BIN,,,,
"(-0.0009999183, 0.217]",20096,4514,0.088862,0.227291
"(0.217, 0.341]",21612,2980,0.095566,0.150050
"(0.341, 0.441]",22156,2446,0.097971,0.123162
"(0.441, 0.513]",22530,2072,0.099625,0.104330
"(0.513, 0.566]",22997,1865,0.101690,0.093907
"(0.566, 0.608]",22741,1596,0.100558,0.080363
"(0.608, 0.646]",23156,1444,0.102393,0.072709
"(0.646, 0.682]",23417,1187,0.103547,0.059768
"(0.682, 0.722]",23565,1033,0.104202,0.052014


In [109]:
#CALCULATE WOE
EXT_SOURCE_2_BIN_CROSSTAB['WOE']=np.log(EXT_SOURCE_2_BIN_CROSSTAB['dist_non_default']/EXT_SOURCE_2_BIN_CROSSTAB['dist_default'])
EXT_SOURCE_2_BIN_CROSSTAB

TARGET,0,1,dist_non_default,dist_default,WOE
EXT_SOURCE_2_BIN,,,,,
"(-0.0009999183, 0.217]",20096,4514,0.088862,0.227291,-0.939145
"(0.217, 0.341]",21612,2980,0.095566,0.150050,-0.451157
"(0.341, 0.441]",22156,2446,0.097971,0.123162,-0.228828
"(0.441, 0.513]",22530,2072,0.099625,0.104330,-0.046149
"(0.513, 0.566]",22997,1865,0.101690,0.093907,0.079621
"(0.566, 0.608]",22741,1596,0.100558,0.080363,0.224187
"(0.608, 0.646]",23156,1444,0.102393,0.072709,0.342355
"(0.646, 0.682]",23417,1187,0.103547,0.059768,0.549551
"(0.682, 0.722]",23565,1033,0.104202,0.052014,0.694813


In [110]:
#CALCULATE IV
EXT_SOURCE_2_BIN_CROSSTAB['IV_COMPONENT']=(EXT_SOURCE_2_BIN_CROSSTAB['dist_non_default']-EXT_SOURCE_2_BIN_CROSSTAB['dist_default'])*EXT_SOURCE_2_BIN_CROSSTAB['WOE']
EXT_SOURCE_2_BIN_CROSSTAB

TARGET,0,1,dist_non_default,dist_default,WOE,IV_COMPONENT
EXT_SOURCE_2_BIN,,,,,,
"(-0.0009999183, 0.217]",20096,4514,0.088862,0.227291,-0.939145,0.130005
"(0.217, 0.341]",21612,2980,0.095566,0.150050,-0.451157,0.024581
"(0.341, 0.441]",22156,2446,0.097971,0.123162,-0.228828,0.005764
"(0.441, 0.513]",22530,2072,0.099625,0.104330,-0.046149,0.000217
"(0.513, 0.566]",22997,1865,0.101690,0.093907,0.079621,0.000620
"(0.566, 0.608]",22741,1596,0.100558,0.080363,0.224187,0.004528
"(0.608, 0.646]",23156,1444,0.102393,0.072709,0.342355,0.010163
"(0.646, 0.682]",23417,1187,0.103547,0.059768,0.549551,0.024059
"(0.682, 0.722]",23565,1033,0.104202,0.052014,0.694813,0.036261


In [111]:
total_iv=EXT_SOURCE_2_BIN_CROSSTAB['IV_COMPONENT'].sum()
total_iv

np.float64(0.3098617800420801)

# CREATING A BINNING FUNCTION

In [112]:
def create_quantile_bins(dataframe,variable, n_bins=10):
    binning_series, bin_edges=pd.qcut(dataframe[variable],q=n_bins,duplicates='drop',retbins=True)
    return binning_series, bin_edges
    

In [113]:
#Testing above function on ext_source_2
train_data["EXT_SOURCE_2_BIN"], ext2_edges=create_quantile_bins(train_data,'EXT_SOURCE_2')

In [114]:
ext2_edges

array([8.17361652e-08, 2.16526383e-01, 3.40682668e-01, 4.41383257e-01,
       5.12939136e-01, 5.66111829e-01, 6.08146663e-01, 6.45843240e-01,
       6.82144661e-01, 7.22087541e-01, 8.54999666e-01])

In [115]:
train_data["EXT_SOURCE_2_BIN"]

0                 (0.341, 0.441]
1                 (0.513, 0.566]
2                 (0.646, 0.682]
3         (-0.0009999183, 0.217]
4                 (0.441, 0.513]
                   ...          
246003            (0.682, 0.722]
246004            (0.608, 0.646]
246005            (0.608, 0.646]
246006            (0.646, 0.682]
246007            (0.566, 0.608]
Name: EXT_SOURCE_2_BIN, Length: 246008, dtype: category
Categories (10, interval[float64, right]): [(-0.0009999183, 0.217] < (0.217, 0.341] < (0.341, 0.441] < (0.441, 0.513] ... (0.608, 0.646] < (0.646, 0.682] < (0.682, 0.722] < (0.722, 0.855]]

# CREATE WOE IV FUNCTION

In [116]:
def calculate_woe_iv(dataframe, binned_col_name, target_variable):
    
    # Step 1: Crosstab
    cross_tab = pd.crosstab(
        index=dataframe[binned_col_name],
        columns=dataframe[target_variable]
    )
    
    # Ensure both 0 and 1 columns exist
    if 0 not in cross_tab.columns:
        cross_tab[0] = 0
    if 1 not in cross_tab.columns:
        cross_tab[1] = 0

    # Step 2: Total goods and bads
    total_good = cross_tab[0].sum()
    total_bad = cross_tab[1].sum()
    
    # Step 3: Distribution
    cross_tab['%good'] = cross_tab[0] / total_good
    cross_tab['%bad'] = cross_tab[1] / total_bad
    
    # Step 4: Replace 0 with small value to avoid division by zero
    epsilon = 1e-6
    cross_tab['%good'] = cross_tab['%good'].replace(0, epsilon)
    cross_tab['%bad'] = cross_tab['%bad'].replace(0, epsilon)
    
    # Step 5: WOE
    cross_tab['WOE'] = np.log(cross_tab['%good'] / cross_tab['%bad'])
    
    # Step 6: IV component
    cross_tab['IV_COMPONENT'] = (
        (cross_tab['%good'] - cross_tab['%bad']) * cross_tab['WOE']
    )
    
    # Step 7: Total IV
    total_iv = cross_tab['IV_COMPONENT'].sum()
    
    return cross_tab, total_iv


In [117]:
ext_source_2_ct, ext_source_2_iv=calculate_woe_iv(train_data,"EXT_SOURCE_2_BIN","TARGET")

In [118]:
ext_source_2_ct

TARGET,0,1,%good,%bad,WOE,IV_COMPONENT
EXT_SOURCE_2_BIN,,,,,,
"(-0.0009999183, 0.217]",20096,4514,0.088862,0.227291,-0.939145,0.130005
"(0.217, 0.341]",21612,2980,0.095566,0.150050,-0.451157,0.024581
"(0.341, 0.441]",22156,2446,0.097971,0.123162,-0.228828,0.005764
"(0.441, 0.513]",22530,2072,0.099625,0.104330,-0.046149,0.000217
"(0.513, 0.566]",22997,1865,0.101690,0.093907,0.079621,0.000620
"(0.566, 0.608]",22741,1596,0.100558,0.080363,0.224187,0.004528
"(0.608, 0.646]",23156,1444,0.102393,0.072709,0.342355,0.010163
"(0.646, 0.682]",23417,1187,0.103547,0.059768,0.549551,0.024059
"(0.682, 0.722]",23565,1033,0.104202,0.052014,0.694813,0.036261


In [119]:
ext_source_2_iv

np.float64(0.3098617800420801)

# LOOP OVER ALL NUMERIC COLS AND RUN BINNING AND WOE functions

In [120]:
#CREATE DICTIONARIES TO STORE
#IV VALUES OF COLS
#WOE TABLES FOR EACH COLS BINS
#BIN EDGES FOR COLS
woe_tables = {}
bin_edges_dict = {}
iv_dict = {}

In [121]:
# NOW WE LOOP OVER EACH COL AND RUN THE FUNCTS
for col in numerical_cols:
    try:
        binned_series,bin_edges=create_quantile_bins(train_data,col)
        temp_df=pd.DataFrame({col+"_BIN":binned_series,"TARGET":train_data["TARGET"]})
        woe_ct, total_iv=calculate_woe_iv(temp_df,col+"_BIN","TARGET")
        woe_tables[col]=woe_ct
        bin_edges_dict[col]=bin_edges
        iv_dict[col]=total_iv
    except Exception as e:
        print(f"{col} had some error {e}, skipping.")

In [122]:
# converting iv diction to dataframe
iv_summary_numeric = pd.DataFrame.from_dict(
    iv_dict,
    orient='index',
    columns=['IV']
).sort_values(by='IV', ascending=False)

In [123]:
iv_summary_numeric.head(10)


,IV
EXT_SOURCE_3,0.310081
EXT_SOURCE_2,0.309862
EXT_SOURCE_1,0.135271
DAYS_EMPLOYED,0.100876
AMT_GOODS_PRICE,0.093900
DAYS_BIRTH,0.083788
AMT_CREDIT,0.047558
DAYS_LAST_PHONE_CHANGE,0.047094
DAYS_ID_PUBLISH,0.039051
REGION_POPULATION_RELATIVE,0.037964


# CREATE WOE AND IVs FOR CATEGORICAL AND MISSING FLAG COLS

In [125]:
print(len(categorical_cols))
len(missing_flag_cols)

16


17

In [126]:
#combine both categorical and missing flag col as missing flag cols are categorical only
categorical_all=categorical_cols+missing_flag_cols
categorical_all

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE',
 'OWN_CAR_AGE_missing_flag',
 'YEARS_BUILD_AVG_missing_flag',
 'COMMONAREA_AVG_missing_flag',
 'FLOORSMIN_AVG_missing_flag',
 'LIVINGAPARTMENTS_AVG_missing_flag',
 'NONLIVINGAPARTMENTS_AVG_missing_flag',
 'YEARS_BUILD_MODE_missing_flag',
 'COMMONAREA_MODE_missing_flag',
 'FLOORSMIN_MODE_missing_flag',
 'LIVINGAPARTMENTS_MODE_missing_flag',
 'NONLIVINGAPARTMENTS_MODE_missing_flag',
 'YEARS_BUILD_MEDI_missing_flag',
 'COMMONAREA_MEDI_missing_flag',
 'FLOORSMIN_MEDI_missing_flag',
 'LIVINGAPARTMENTS_MEDI_missing_flag',
 'NONLIVINGAPARTMENTS_MEDI_missing_flag',
 'FONDKAPREMONT_MODE_missing_flag']

In [127]:
#Loop through each categorical col and create woe and iv(BINNING NOT REQUIRED HERE)
for col in categorical_all:
    try:
        woe_ct, total_iv = calculate_woe_iv(train_data, col, "TARGET")
        woe_tables[col]=woe_ct
        iv_dict[col]=total_iv
    except Exception as e:
        print(f"{col} variable has a an error {e}. Skipping")

In [132]:
iv_summary_all = pd.DataFrame.from_dict(
    iv_dict,
    orient='index',
    columns=['IV']
).sort_values(by='IV', ascending=False)

In [133]:
pd.set_option('display.max_rows',10000)
iv_summary_all.head(15)

,IV
EXT_SOURCE_3,0.310081
EXT_SOURCE_2,0.309862
EXT_SOURCE_1,0.135271
DAYS_EMPLOYED,0.100876
AMT_GOODS_PRICE,0.093900
OCCUPATION_TYPE,0.087032
DAYS_BIRTH,0.083788
ORGANIZATION_TYPE,0.075240
NAME_INCOME_TYPE,0.059788
NAME_EDUCATION_TYPE,0.054864


# NOW VARIABLE ELIMINATION BASED ON IV

In [134]:
type(iv_summary_all)

pandas.core.frame.DataFrame

In [139]:
#taking IV Cutoff as 0.05
selected_vars=iv_summary_all[iv_summary_all['IV']>=0.05].index.tolist()

In [140]:
len(selected_vars)

10

In [141]:
#Final DF after woe and iv
train_model_base=train_data[selected_vars + ["TARGET"]]

In [143]:
#EXPORT MODEL BASE TO PROCESSED DATA SET
train_model_base.to_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\train_model_base.csv",index=False)

# EXPORT THE DICTIONARIES AND SELECTED VARIABLES
1) SELECTED VARS
2) WOE TABLES
3) BIN EDGES DICTIONARY

In [145]:
import os

os.makedirs("models", exist_ok=True)

In [146]:
import pickle

model_artifacts = {
    "selected_vars": selected_vars,
    "woe_tables": woe_tables,
    "bin_edges_dict": bin_edges_dict
}

with open("models/model_artifacts.pkl", "wb") as f:
    pickle.dump(model_artifacts, f)